## 2360. Longest Cycle in a Graph

# Intuition
<!-- Describe your first thoughts on how to solve this problem. -->
There are only 4 cases of the pattern:
- 1 linear structure

![image.png](https://assets.leetcode.com/users/images/b6c26d03-a976-4b1d-9eba-98d9cd082644_1682744764.2557895.png)
- 2 circle

![image.png](https://assets.leetcode.com/users/images/c37ad03f-e4b6-4638-95e6-78ad620455e0_1682744867.8634007.png)
- 3 circle with branches

![image.png](https://assets.leetcode.com/users/images/ea318fdf-f6d1-43d4-93ba-21df61f245fe_1682744920.6607926.png)

- 4 combinations of the (1, 2) and (1, 3), as (2, 3) is not possible(outgoing degree at most 1, otherwise should be more difficult)

- If we can find a way to trim all branches of the circle, we can get the result using bfs to get the longest.
- To trim the branches, we can use topological sort as the branch structure is a topological graph.
- we can start form indegree = 0 to trim the graph grudaully, until there is only circle.

# Approach
<!-- Describe your approach to solving the problem. -->
- 1 get indgrees all nodes
- 2 trim nodes with 0 indegree iteratively to make a circle (topological sort: bfs)
- 3 find the longest cycle amoung all cycles after trimming 0 indgree nodes (bfs)
# Complexity
- Time complexity:
<!-- Add your time complexity here, e.g. $$O(n)$$ -->
O(n)
- Space complexity:
<!-- Add your space complexity here, e.g. $$O(n)$$ -->
O(n)

In [1]:
class Solution:
    def longestCycle(self, edges) -> int:
        n = len(edges)
        indegree = [0 for i in range(n)]

        # 1 get indgrees all nodes
        for u in edges:
            if u == -1:
                continue
            indegree[u] += 1

        # 2 trim nodes with 0 indegree iteratively to make a circle (topological sort: bfs)
        for u in range(n):
            while indegree[u] == 0 and edges[u] != -1:
                v = edges[u]
                edges[u] = -1
                indegree[v] -= 1
                u = v
        
        # 3 find the longest cycle amoung all cycles after trimming 0 indgree nodes (bfs)
        visited, res = set(), -1
        for u in range(n):
            if indegree[u] == 0 or u in visited:
                continue
            # bfs to find length
            ans = 1
            visited.add(u)
            while edges[u] not in visited:
                v = edges[u]
                ans += 1
                u = v
            res = max(res, ans)
        return res

In [3]:
class Solution:
    def longestCycle(self, edges) -> int:
        n, res, visited = len(edges), -1, set()
        indegrees = [0] * n

        for u in edges:
            if u == -1:
                continue
            indegrees[u] += 1
        
        for u in range(n):
            while indegrees[u] == 0 and edges[u] != -1:
                v = edges[u]
                indegrees[v] -= 1
                edges[u] = -1
                u = v
        
        for u in range(n):
            if edges[u] == -1 or edges[u] in visited:
                continue
            ans = 0
            while edges[u] not in visited:
                visited.add(edges[u])
                v = edges[u]
                ans += 1
                u = v
            res = max(res, ans)

        return res

## 2127. Maximum Employees to Be Invited to a Meeting

# Intuition
<!-- Describe your first thoughts on how to solve this problem. -->
2360. Longest Cycle in a Graph
- https://leetcode.com/problems/longest-cycle-in-a-graph/description/
# Approach
<!-- Describe your approach to solving the problem. -->
- Except the above patterns, need to consider another case:
  - loops of 2 node with edges, all these cases can add up together to meet the sitting table conditoion
![image.png](https://assets.leetcode.com/users/images/a2997a16-cd7a-4aae-9640-7a01905b625f_1682758952.7499888.png)
  - one trick is: depth[v] =max(depth[v], 1 + depth[u]) # it can be itself or prev + 1
  - while bfs, need to record the depth array, current node should consider the previou node depth
  - Another trick is to change the final bfs structure: visited.add position is changed to void if a loop like 1-2-3-4-1, if use the leetcode 2360 method, it will count to 4-1 case as a loop of 2, which will got the wrong result. This means leetcode 2360's final bfs can be improved
  
```
for u in range(n):
    if indegree[u] == 0 or u in visited:
        continue
    # bfs to find length
    ans = 0
    while favorite[u] not in visited:
        visited.add(favorite[u])
        v = favorite[u]
        ans += 1
        u = v
    if ans == 2:
        total_link_loop_2 += depth[u] +depth[favorite[u]]
    elif ans >= 3:
        res = max(res, ans)
return max(res, total_link_loop_2)
```

# Complexity
- Time complexity:
<!-- Add your time complexity here, e.g. $$O(n)$$ -->
O(n)
- Space complexity:
<!-- Add your space complexity here, e.g. $$O(n)$$ -->
O(n)


In [ ]:
class Solution:
    def maximumInvitations(self, favorite: List[int]) -> int:
        n = len(favorite)
        indegree = [0 for i in range(n)]

        # 1 get indgrees of all nodes
        for u in favorite:
            if u == -1:
                continue
            indegree[u] += 1

        # 2 trim nodes with 0 indegree iteratively to make a circle (topological sort: bfs)
        # and calculate the depth of all node at the same time
        depth = [1] * n
        for u in range(n):
            temp = 1
            while indegree[u] == 0 and favorite[u] != -1:
                v = favorite[u]
                favorite[u] = -1
                indegree[v] -= 1
                depth[v] =max(depth[v], 1 + depth[u]) # it can be itself or prev + 1
                u = v
    
        # 3 find the longest cycle amoung all cycles after trimming 0 indgree nodes (bfs)
        # if the circle lenght is more than 3, just use max, if it is 2 use total to add all of them
        visited, res, total_link_loop_2 = set(), 0, 0
        for u in range(n):
            if indegree[u] == 0 or u in visited:
                continue
            # bfs to find length
            ans = 0
            while favorite[u] not in visited:
                visited.add(favorite[u])
                v = favorite[u]
                ans += 1
                u = v
            if ans == 2:
                total_link_loop_2 += depth[u] +depth[favorite[u]]
            elif ans >= 3:
                res = max(res, ans)
        return max(res, total_link_loop_2)